In [ ]:
import datetime
import gzip
import json
import time
import os

import dask
from dask.distributed import Client

client = Client("tls://localhost:8786")

In [ ]:
fname = "../ntuple_production/file_metadata.json.gz"
with gzip.open(fname) as f:
    dataset_info = json.loads(f.read().decode())

all_files = []
for containers_for_category in dataset_info.values():
    for container, metadata in containers_for_category.items():
        if metadata["files_output"] is None:
            continue

        all_files += metadata["files_output"]

len(all_files)

In [ ]:
def run_xrdcp(fname):
    t0 = time.perf_counter()
    os.system(f"xrdcp {fname} /dev/null -f")
    t1 = time.perf_counter()
    time_finished = datetime.datetime.now()
    return {"runtime": t1-t0, "time_finished": time_finished}


tasks = [dask.delayed(run_xrdcp)(fname) for fname in all_files]
_ = dask.compute(*tasks)